**{DEV} TODO:**
- Terminology
- Jacobian: here or 1b

---
### Chapter 1: System Modelling
---
# **Tutorial 1.a: Lagrange Mechanics of a Pendulum**
**Aim:** To derive the dynamic model of a double pendulum using the Lagrange method using Python's symbolic toolbox.

**Further watching:** <a href="https://www.youtube.com/watch?v=zhk9xLjrmi4"> MIT OCW lecture on Lagrange mechanics with examples. </a>
### **Contents**:
* [Chain Systems](#Chain-Systems)
* [Coordinate Frames](#Coordinate-Frames)
* [Lagrange Mechanics](#Lagrange-Mechanics)
* [Simulation](#Simulation)

## **Chain Systems**
We model robots as **chains** of rigid **links**. If the links follow one after the other, it's a serial chain (e.g. upper arm -> forearm). If there are branches where more than one *child* link follows from a single *parent* link, it's a parallel chain (e.g. torso -> left thigh OR right thigh).

Most of the systems we model will be open chains, i.e. they don't involve any loops. If two parallel branches connect together, they form a closed chain (e.g. a four bar linkage system, like Baleka's leg).

<img src = "chain2.png" width = "600">

### **Terminology:**
- **Chain** - a group of links that are attached to each other
- **Link** - a single rigid body within a chain system
- **Serial Chain** - a chain where each link attaches to the previous link
- **Parallel Chain** - a chain where multiple child links originate froma single parent link
- **Closed Chain** - parallel chains that join at both ends, forming a loop
- **Open Chain** - parallel chains that only join at the parent link

## **Coordinate Frames**
A coordinate *frame* consists of an origin and three perpendicular axes (\*if working in 3D).

In order to completely describe the location of an individual link in 3D space, you need six coordinates: the position $x,y,z$, and the orientation $\alpha,\beta,\gamma$ (the roll, pitch and yaw angles about these axes).

The **system space** is the set of all $6n$ coordinates you need to completely define the *pose* of an $n$-link system, when the position of all links is described *absolutely* (i.e. relative to the fixed world frame). This is a *maximal* coordinate system, since it needs the maximum number of coordinates.

An alternative approach—the one used with Lagrange mechanics—is to use a smaller system of *generalized coordinates*. For chain systems, these typically correspond to the **joint space** of the system: the positions of all the joints between links. These are relative coordinates: rather than describing the links in the world frame, the position of the next link is defined from the previous link.

<img src = "coordinate_systems.png" width = "400">

The minimum number of generalized coordinates you need to *completely and uniquely* describe the pose gives the number of *degrees of freedom (DOFs)* of the system.

### The Double Pendulum
We'll be generating the dynamic model for the 2D double pendulum shown in the previous image.

It is a 2 DOF system, since it can be defined completely by the two joint angles, $\theta_1$ and $\theta_2$. (This is a *fixed base* system, since the first link is constrained such that it can only rotate. If it was a *floating base* system, where this link was free to move in all directions, we'd need an extra 2 DOFs—$x$ and $y$—to completely describe the pose.)

Our generalized coordinate vector is therefore:
$$\mathbf{q} = \begin{pmatrix} \theta_1 \\ \theta_2 \end{pmatrix}$$

### **Terminology:** {DEVNOTE: terminology still WIP}
- **System Space** - maximal coordinate system, describing the position and orientation of each link relative to the world frame
- **Joint Space** - coordinate system that describes the location of each link relative to its parent link
- **Generalized Coordinates** - the smaller set of coordinates which are used when describing locations in joint space

In [ ]:
# import libraries
import sympy as sym
import numpy as np

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>")) #ignore this, it'll be useful in tut 3

In [ ]:
# create symbolic variables

# system parameters
g = sym.symbols('g')
m1 ,  m2 = sym.symbols([ 'm_{1}', 'm_{2}']) # mass of links
l1 ,  l2 = sym.symbols([ 'l_{1}', 'l_{2}']) # length of links
In1, In2 = sym.symbols(['In_{1}','In_{2}']) # moment of intertia of links

# generalized coordinates
X0, Y0 = sym.symbols(['X_{0}','Y_{0}']) # fixed position of first link

th1  ,  th2 = sym.symbols([       '\\theta_{1}',       '\\theta_{2}']) #positions
dth1 , dth2 = sym.symbols([ '\dot{\\theta}_{1}', '\dot{\\theta}_{2}']) #velocities
ddth1,ddth2 = sym.symbols(['\ddot{\\theta}_{1}','\ddot{\\theta}_{2}']) #accelerations

q   = sym.Matrix([  [th1],  [th2]]) #group into matrices
dq  = sym.Matrix([ [dth1], [dth2]])
ddq = sym.Matrix([[ddth1],[ddth2]])

display(ddq) #display prints it as cool latex stuff

\***NOTE:**
- You need to use sympy-specific versions of functions (like the trig ones) if you want to take the derivatives later and whatnot.
- Above, it was stated that you need 6 coordinates to completely describe the location of a link **in 3D**. In 2D, you need 3: the position $x,y$, and the orientation $\theta$. In other words, the 2D **system space** comprises $3n$ coordinates for an n-link system.

In [ ]:
# STEP 1: system space coordinates written in terms of the generalised coordinates

# helper functions
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def GetXY(v):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v.multiply_elementwise(vector_mask)

# positions of each link in their own reference frames
r0 = sym.Matrix([[X0],[Y0],[0]]) # position of the origin of the first link
r1_1 = sym.Matrix([[0],[-0.5*l1],[0]]) # read as: position r1, in frame 1
r2_2 = sym.Matrix([[0],[-0.5*l2],[0]]) # both point vertically down, with centres of mass halfway down the total length of the link

# positions of each link, moved into the inertial frame
r1_0 = Rotate(r1_1, th1) + GetXY(r0)

r2_1 = Rotate(r2_2, th2) + GetXY(r1_1)
r2_0 = Rotate(r2_1, th1) + GetXY(r1_0) # this code is improved upon in tut 3

display(r1_0)
display(r2_0)

\***NOTE:** The **Jacobian** is explained in tut 1b. For now, just think of it as a fancy derivative.

In [ ]:
# STEP 2: generate expressions for the system space velocities

dr1 = r1_0.jacobian(q)*dq
dr2 = r2_0.jacobian(q)*dq

display(dr1)
display(dr2) # just to show that it works

## **Lagrange Mechanics**
Lagrange mechanics is just a reformulation of classical mechanics around this guy, the Lagrangian: 
$$L = T - V$$
where $T$ is the kinetic energy of the system and $V$ is the potential energy.

The only equation you really need to know to get your dynamic model out is: 
$$\frac{d}{dt}\left(\frac{\partial L}{\partial \dot{q}_{i}}\right) - \frac{\partial L}{\partial q_i} = 0$$ 

\***Note:** this is only for the case where there are no external forces acting. We'll deal with those just now.

I find it useful to expand that equation into four terms:
$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}_{i}}\right)
-\frac{d}{dt}\left(\frac{\partial V}{\partial \dot{q}_{i}}\right)
-\frac{\partial T}{\partial q_i}
+\frac{\partial V}{\partial q_i}=0$$

In our applications, we're unlikely to come up against any situation where the potential energy is velocity-dependent so you can almost certainly assume that the second term will be zero.

You need one equation per coordinate. If you were doing it by hand, you'd write out that equation for each coordinate $i$ in your $q$ vector.

If you're more familiar with the manipulator equation, you may note that from the above equation:
<!-- $$\mathbf{M}(q)\cdot \ddot{q} + \mathbf{\dot{M}}(q)\cdot \dot{q} = \frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}}\right)
\hspace{2 em}\&\hspace{2 em} \mathbf{C}(q,\dot{q})\cdot \dot{q} = \mathbf{\dot{M}}(q)\cdot \dot{q} -\frac{\partial T}{\partial q}
\hspace{2 em}\&\hspace{2 em} \mathbf{G}^{T}(q) = \frac{\partial V}{\partial q}$$ -->

$$\color{blue}{\mathbf{M}(q)\cdot \ddot{q}+\mathbf{\dot{M}}(q)\cdot \dot{q}} = \color{red}{\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}}\right)}$$
$$\color{blue}{\mathbf{C}(q,\dot{q})\cdot \dot{q}} = \color{blue}{\mathbf{\dot{M}}(q)\cdot \dot{q}} - \color{red}{\frac{\partial T}{\partial q}}$$
$$\color{blue}{\mathbf{G}^{T}(q)} = \color{red}{\frac{\partial V}{\partial q}}$$

$$\color{blue}{\therefore \mathbf{M}(q)\cdot \ddot{q} + \mathbf{C}(q,\dot{q})\cdot \dot{q} + \mathbf{G}^{T}(q)}
= \color{red}{\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}_{i}}\right)
-\frac{\partial T}{\partial q_i}
+\frac{\partial V}{\partial q_i}}$$
Thus, these two forms are equivalent.

In [ ]:
# STEP 3: generate expressions for the kinetic and potential energy

# helper functions
def Ek(m, In, dr):
    InM = sym.Matrix([[m,0,0],[0,m,0],[0,0,In]])
    return 0.5*dr.T*InM*dr

def Ep(m, r):
    return sym.Matrix([m*g*r[1]])

T = Ek(m1, In1, dr1) + Ek(m2, In2, dr2)
V = Ep(m1, r1_0) + Ep(m2, r2_0)

T = sym.trigsimp(T)
V = sym.trigsimp(V)
display(T)
display(V)

In [ ]:
# STEP 4: calculate each term of the Lagrange equation

# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 2 = 0
# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

#combine
EOM = Lg1 - Lg3 + Lg4

EOM = EOM.T
EOM = sym.trigsimp(EOM) # <- COMMENT OUT THIS LINE IF IT IS TOO SLOW
display(EOM) # <- this should give you an idea of the magnitude of the faff a decent symbolic package lets you avoid

## **Simulation**
To test if the equations work, we can use them to simulate the dynamics forward from some initial condition, e.g. rest at the position $\theta_1 = \pi/2$, $\theta_2 = 0$.

### Approximate integration with implicit Euler
The derivative of a variable can be approximated by the expression:
$$\frac{dx[t]}{dt} = \dot{x}[t] \approx \frac{x[t]-x[t-1]}{h},$$
where $h$ is the timestep between the instances $t-1$ and $t$.

Rearranging this lets us calculate the value of the variable at the current time by approximately integrating from the previous instant:
$$x[t] = x[t-1]+h\dot{x}[t]$$

\***NOTE:** Implicit Euler has better stability than explicit Euler. Make sure to use $h\dot{x}[t]$ and not $h\dot{x}[t-1]$!

In [ ]:
h = 0.01

#parameters
X0val, Y0val  = 0, 2
l1val, l2val  = 1, 1

parameter_values = [(X0,X0val),(Y0,Y0val),
                    (g,9.81),
                    (m1,1),(m2,1),
                    (l1,l1val),(l2,l2val),
                    (In1,0.08),(In2,0.08)]

# substitute parameters into EOM
EOM_sub_params = EOM.subs(parameter_values)

#initial conditions
th1vals  = [np.pi/4]
th2vals  = [0]
dth1vals = [0]
dth2vals = [0]

N = 100
for i in range(1,N+1):
    # conditions at previous timestep
    prev = [( th1,  th1vals[i-1]),
            ( th2,  th2vals[i-1]),
            (dth1, dth1vals[i-1]),
            (dth2, dth2vals[i-1])]
    
    # substitute previous conditions into EOM
    EOM_sub = EOM_sub_params.subs(prev)
    
    # solve for the acceleration
    accel = sym.solve(EOM_sub,[ddth1, ddth2])
    
    # integrate for the next velocity  
    dth1vals.append(float(dth1vals[i-1] + h * accel[ddth1]))
    dth2vals.append(float(dth2vals[i-1] + h * accel[ddth2]))
    
    # integrate for the next position
    th1vals.append(float(th1vals[i-1] + h * dth1vals[i]))
    th2vals.append(float(th2vals[i-1] + h * dth2vals[i]))
    
    print('\r%s' %i ,end='') # <- so you know how far into the simulation you are (for peace of mind)

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_pendulum(i,th1_in,th2_in,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-2,2])
    ax.set_ylim([0,4])
    
    #plot link 1
    L1topx = X0val
    L1topy = Y0val
    L1bottomx = X0val + l1val*np.sin(th1_in[i])
    L1bottomy = Y0val - l1val*np.cos(th1_in[i])
    ax.plot([L1topx,L1bottomx],[L1topy,L1bottomy],color='xkcd:blue')
    
    #plot link 2
    L2bottomx = L1bottomx + l2val*np.sin(th1_in[i] + th2_in[i])
    L2bottomy = L1bottomy - l2val*np.cos(th1_in[i] + th2_in[i]) 
    ax.plot([L1bottomx,L2bottomx],[L1bottomy,L2bottomy],color='xkcd:red')
    
update = lambda i: plot_pendulum(i,th1vals,th2vals,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(N),interval = 50,repeat=True) # interval = frame time. 1/50 = 20 fps
# animate = ani.FuncAnimation(fig1,update,range(N),interval = 1000*h,repeat=True) # if you want it to play at the actual speed

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
# ani.Animation.save(animate,'pendulum_swing.mp4', fps=int(1/h), dpi=300) # if you want to save the animation instead of embedding it